In [ ]:
import numpy as np
import pandas as pd
from aeon.datasets import load_classification
import numpy as np
from SeqDT import *
from Representation import *
from ModelEvaluation import *
from Discretization import *
from TimeSeriesAnalysis import *
from aeon.datasets.dataset_collections import get_available_tsc_datasets

In [ ]:
patiens = [4, 6]

## Dataset AEON

In [ ]:
available_datasets = get_available_tsc_datasets()

ecg_related = [d for d in available_datasets if 'ECG' in d or 'ecg' in d.lower()]

print("Dataset ECG available:")
for dataset in ecg_related:
    print(f"  - {dataset}")

## Dataset ECG 200

In [ ]:
X_train, y_train = load_classification("ECG200", split="train")
X_test, y_test = load_classification("ECG200", split="test")

print(f"Training set: {X_train.shape}")
print(f"Classes: {np.unique(y_train)}")

In [ ]:
type(X_train)

In [ ]:
ECG200 = TSAnalysis(X_train, y_train, X_test, y_test, bins = 5, method= 'Classic', patients_to_viz= patiens)

In [ ]:
type(ECG200)

In [ ]:
ECG200_RLR = TSAnalysis(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, bins = 5, method= 'RLR', patients_to_viz= patiens)

## ECG datasets

In [ ]:
Summary = pd.DataFrame()

In [ ]:
for dataset in ecg_related:
    X_train, y_train = load_classification(dataset, split="train")
    X_test, y_test = load_classification(dataset, split="test")

    print(f'\n\nDATASET: {dataset}')
    print(f"Training set: {X_train.shape}")
    print(f"Classes: {np.unique(y_train)}")
    
    result = TSAnalysis(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, bins = 5, method= 'Classic', dataset_name= dataset)
    
    if Summary.empty:
        Summary = pd.DataFrame([result])
    else:
        Summary = pd.concat([Summary, pd.DataFrame([result])], ignore_index=True)
        
        
    result = TSAnalysis(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, bins = 5, method= 'RLR', dataset_name= dataset)
    
    Summary = pd.concat([Summary, pd.DataFrame([result])], ignore_index=True)

In [ ]:
Summary

# Parameters

In [ ]:
params_conservative = {
    'g': 1,
    'maxL': 2,
    'pru': True,
    'epsilon': 0.2,
    'minS': 0.01,
    'minN': 5,
    'maxD': 3,
    'visualize': False,
    'show_statistics': False
}

params_standard = {
    'g': 1,
    'maxL': 3,
    'pru': True,
    'epsilon': 0.15,
    'minS': 0,
    'minN': 2,
    'maxD': 0,
    'visualize': False,
    'show_statistics': False
}

params_aggressive = {
    'g': 1,
    'maxL': 4,
    'pru': True,
    'epsilon': 0.1,
    'minS': 0,
    'minN': 1,
    'maxD': 0,
    'visualize': False,
    'show_statistics': False
}

params_high_gap = {
    'g': 2,
    'maxL': 3,
    'pru': True,
    'epsilon': 0.15,
    'minS': 0,
    'minN': 2,
    'maxD': 0,
    'visualize': False,
    'show_statistics': False
}

params_no_pruning = {
    'g': 1,
    'maxL': 3,
    'pru': False,
    'epsilon': 0.15,
    'minS': 0,
    'minN': 2,
    'maxD': 0,
    'visualize': False,
    'show_statistics': False
}

params_shallow = {
    'g': 1,
    'maxL': 3,
    'pru': True,
    'epsilon': 0.15,
    'minS': 0,
    'minN': 2,
    'maxD': 2,
    'visualize': False,
    'show_statistics': False
}



parameter_configs = {
    'Conservative': params_conservative,
    'Standard': params_standard,
    'Aggressive': params_aggressive,
    'HighGap': params_high_gap,
    'NoPruning': params_no_pruning,
    'Shallow': params_shallow
}


In [ ]:
bins_configs = [3, 5, 7, 9]

methods = ['Classic', 'RLR']

In [ ]:
Summary = pd.DataFrame()
All_Results = pd.DataFrame()  # Salva tutti i risultati per confronto

In [ ]:
for dataset in ecg_related:
    try:
        X_train, y_train = load_classification(dataset, split="train")
        X_test, y_test = load_classification(dataset, split="test")

        print(f'\n{"="*80}')
        print(f'DATASET: {dataset}')
        print(f"Training set: {X_train.shape}")
        print(f"Classes: {np.unique(y_train)}")
        print(f'{"="*80}\n')
        
        best_accuracy_for_dataset = 0
        best_result_for_dataset = None
        
        
        for bins in bins_configs:
            for param_name, params in parameter_configs.items():
                
                for method in methods:
                    try:
                        print(f"Testing: bins={bins}, params={param_name}, method={method}")
                        
                        result = TSAnalysis(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
                                            bins=bins, method=method, dataset_name=dataset,SeqDT_parameters=params,
                                            visualize_plots= False)
                        
                        result['Param_Config'] = param_name
                        
                        All_Results = pd.concat([All_Results, pd.DataFrame([result])], ignore_index=True)
                        
                        if result['Accuracy'] > best_accuracy_for_dataset:
                            best_accuracy_for_dataset = result['Accuracy']
                            best_result_for_dataset = result.copy()
                        
                    except Exception as e:
                        print(f"Error: {e}")
                
        
        
        if best_result_for_dataset is not None:
            print(f"\nBest result for {dataset}: Accuracy={best_accuracy_for_dataset:.4f}")
            Summary = pd.concat([Summary, pd.DataFrame([best_result_for_dataset])], ignore_index=True)
        
    except Exception as e:
        print(f"\nCould not process dataset {dataset}: {e}")
        continue



In [ ]:

print(f'\n{"="*80}')
print("BEST RESULTS SUMMARY")
print(f'{"="*80}\n')

display_cols = ['Dataset', 'Bins', 'Method', 'Param_Config', 'Accuracy', 'G-mean', 'Tree_Depth', 'Training_Time']
print(Summary[display_cols].to_string(index=False))
print(Summary)
